In [1]:
import pandas as pd
import numpy as np
import png
import time

Load DataFrame of movie scores

`test_data_2.csv`: 171 movies<br>
Full dataset: 17,770 movies

In [2]:
df=pd.read_csv("test_data_2.csv")
df.head()

,movie_id,user_id,rating
0,4500,2532865,4
1,4500,573364,3
2,4500,1696725,3
3,4500,1253431,3
4,4500,1265574,2


Load table of movie titles

In [3]:
titles_df = pd.read_csv("../flask/static/movie_titles.csv",header = None, names = ['movie_id', 'year', 'title'], usecols=[0,1,2],encoding = "ISO-8859-1")
titles_df.head()

,movie_id,year,title
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW


Get range of ids

In [4]:
max_id = max(df["movie_id"])
min_id = min(df["movie_id"])

Split `df` into smaller dataframes by movie, to expedite grid-calculations

In [17]:
movies = {}
ids = list(df["movie_id"])

prev_id, c, c0 = 0, 0, 0

for i in ids:
    if i != prev_id:
        movies[prev_id]=df.iloc[c0:c].drop(["movie_id"],axis=1).set_index("user_id")
        prev_id = i
        c0 = c
    c += 1
movies[ids[-1]]=df.iloc[c0:].drop(["movie_id"],axis=1).set_index("user_id")

Older version using `loc` (much slower when dealing with large df's)

Movies by viewership

In [84]:
df["movie_id"].value_counts()

4656    130782
4640    122158
4577    106291
4590     78719
4545     70799
4661     51392
4670     49457
4633     41850
4506     33731
4520     26500
4660     25232
4546     23270
4570     20703
4652     14966
4586     14888
4612     12453
4628      8519
4517      8440
4669      6989
4533      6857
4627      6432
4634      6409
4557      6294
4565      5979
4508      5939
4524      5792
4658      5741
4522      5736
4591      5246
4568      5174
         ...  
4611       142
4597       136
4514       131
4642       130
4603       130
4589       129
4540       125
4560       124
4538       124
4576       122
4539       122
4510       119
4604       119
4593       119
4620       117
4542       116
4526       116
4664       113
4502       104
4571       104
4637        97
4619        95
4567        92
4573        92
4532        88
4665        87
4584        68
4574        62
4605        59
4614        33
Name: movie_id, Length: 171, dtype: int64

Movies by average rating

In [85]:
df.groupby("movie_id")["rating"].mean().sort_values(ascending=False)

movie_id
4606    4.237500
4633    4.146189
4640    4.125280
4531    4.081783
4509    4.033784
4658    4.016199
4592    4.013333
4628    3.977814
4626    3.976687
4561    3.975752
4521    3.965696
4575    3.960485
4545    3.943968
4506    3.923898
4559    3.914393
4651    3.891344
4585    3.882353
4663    3.877616
4650    3.863158
4577    3.857711
4648    3.840050
4522    3.839784
4595    3.813458
4656    3.813453
4653    3.793714
4525    3.791434
4581    3.775201
4528    3.772039
4508    3.731436
4547    3.728906
          ...   
4639    2.746622
4589    2.728682
4603    2.715385
4539    2.672131
4540    2.656000
4604    2.655462
4537    2.646667
4637    2.618557
4556    2.609756
4622    2.606509
4615    2.568493
4530    2.528796
4532    2.511364
4645    2.500000
4571    2.500000
4611    2.492958
4505    2.470588
4621    2.441030
4582    2.371179
4514    2.358779
4664    2.345133
4538    2.322581
4638    2.228464
4667    2.160839
4502    2.144231
4654    2.000000
4567    1.880435
4584 

In [86]:
def get_title(df, movie_id):
    return df[df.movie_id==movie_id].iloc[0].title

`rough_means_v1`: Outputs list of 3 floats representing normalized (0-1) rating predictions (of `id2`) for high, medium, and low input ratings (of `id1`).<br>
* `col_norm`: How many data points are needed per rating to output the full score. Scores less than this will be scaled down in proportion to number of records. <br>
* `low_clip`: Ignores movie pairs below a certain number of views. Saves time and storage space - recommend at least `30`. Efficiency seems to max out around `150`.

In [87]:
def rough_means_v1(df, id1, id2, movie_dict=None, col_norm=50, low_clip=0, sym=False):
    if id1==id2:
        if sym:
            return [0,0,0],[0,0,0]
        else:
            return [0,0,0]
        
    if movie_dict is None:
        df1 = df[df["movie_id"]==id1].drop(["movie_id"],axis=1)
        df2 = df[df["movie_id"]==id2].drop(["movie_id"],axis=1)
    else:
        df1 = movie_dict[id1]
        df2 = movie_dict[id2]
    
    df_join = df1.join(df2, how="inner", lsuffix="_x", rsuffix="_y")
    
    if sym:
        return rough_means_df(df_join, True, col_norm, low_clip), rough_means_df(df_join, False, col_norm, low_clip)
    else:
        return rough_means_df(df_join, True, col_norm, low_clip)

In [88]:
def rough_means_df(df_join, col_x=True, col_norm=50, low_clip=0):
    total = len(df_join)
    if total == 0 or total < low_clip:
        return [0,0,0]
    
    rx = df_join["rating_x"] if col_x else df_join["rating_y"]
    ry = df_join["rating_y"] if col_x else df_join["rating_x"]
    
    low = df_join[rx<3]
    med = df_join[rx==3]
    high = df_join[rx>3]
    
    result = []
    
    for x in [(low,2),(med,1),(high,2)]:
        if len(x[0])==0:
            result.append(0)
        else:
            if col_x:
                result.append(np.interp(x[0]["rating_y"].mean(),[1,5],[0,1])*np.interp(len(x[0]),[0,col_norm*x[1]],[0,1]))
            else:
                result.append(np.interp(x[0]["rating_x"].mean(),[1,5],[0,1])*np.interp(len(x[0]),[0,col_norm*x[1]],[0,1]))

    return result

`rough_means_matrix`: Calculates `rough_means_v1` for range of values

In [89]:
def rough_means_matrix(df, min_id, max_id, movie_dict=None, col_norm=50, low_clip=0, progress=False, prog_interval=100):
    _range=max_id-min_id+1
    
    matrix = [[[0,0,0]]*_range for x in range(_range)]

    counter=0
    t0=time.time()
    for i in range(min_id, max_id):
        for j in range(i+1, max_id+1):
            x1, x2 = rough_means_v1(df, i, j, movie_dict, col_norm, low_clip, True)
            matrix[i-min_id][j-min_id] = x1
            matrix[j-min_id][i-min_id] = x2
            
            if progress:
                if counter%prog_interval==0:
                    t1=time.time()
                    print(f"{counter} records complete... ({round(t1-t0,3)} s elapsed)")
                counter+=1
    return matrix

Returns the number of viewers in common for two movies

In [90]:
def views(df, id1, id2, movie_dict=None):
    if movie_dict is None:
        df1 = df[df["movie_id"]==id1].drop(["movie_id"],axis=1)
        df2 = df[df["movie_id"]==id2].drop(["movie_id"],axis=1)
    else:
        df1 = movie_dict[id1]
        df2 = movie_dict[id2]
    
    df_join = df1.join(df2, how="inner", lsuffix="_x", rsuffix="_y")
    
    return len(df_join)

In [91]:
def views_matrix(df, min_id, max_id, movie_dict=None,progress=False, prog_interval=100):
    matrix = []
    counter=0
    t0=time.time()
    for i in range(min_id, max_id+1):
        matrix.append([])
        for j in range(min_id, i):
            matrix[i-min_id].append(views(df, i, j, movie_dict))
            
            if progress:
                if counter%prog_interval==0:
                    t1=time.time()
                    print(f"{counter} records complete... ({round(t1-t0,3)} s elapsed)")
                counter+=1
    return matrix

`get_predictions`: Input list of movie-rating pairs (e.g. `[[4508,4],[4602,1],[4501,3],[4655,2]]`) to simulate a new user.<br>
Works with a `matrix` output by `rough_means_v1`

In [92]:
def get_predictions(input_list, matrix, min_id, title_ref=None):
    numrows = len(matrix)
    numcols = numrows
    predictions = []
    
    for m_r in input_list:
        movie_id = m_r[0]
        rating = m_r[1]
        
        predict_row = matrix[movie_id-min_id]
        predict_list = [np.interp(rating,[1,3,5],predict_row[i])*4+1 for i in range(numcols)]
        predictions.append(predict_list)
        
    predict_sums=[]
    for i in range(len(predictions[0])):
        predict_sums.append(0)
        for j in range(len(predictions)):
            predict_sums[i] += predictions[j][i]
    
    predictions_df = pd.DataFrame({"movie_id":range(min_id,min_id+numcols),"rating":[x/len(input_list) for x in predict_sums]})
    predictions_df.query(f'movie_id not in {[x[0] for x in input_list]}', inplace=True)
    if title_ref is not None:
        predictions_df["title"]=[get_title(title_ref, i) for i in predictions_df["movie_id"]]
    
    return predictions_df.sort_values("rating",ascending=False)

Run predictions while ignoring movie pairs below a certain number of views

In [93]:
def get_predictions_clip(input_list, matrix, min_id, views_matrix, low_clip=30, title_ref=None):
    numrows = len(matrix)
    numcols = numrows
    predictions = []
    
    for m_r in input_list:
        movie_id = m_r[0]
        rating = m_r[1]
        
        id1 = movie_id-min_id
        predict_row = []
        for id2 in range(numcols):
            if id2 < id1:
                v = views_matrix[id1][id2]
            elif id2 > id1:
                v = views_matrix[id2][id1]
            else:
                v = 0
            if v < low_clip:
                x = [0,0,0]
            else:
                x = matrix[id1][id2]
            predict_row.append(x)
        
        predict_list = [np.interp(rating,[1,3,5],predict_row[i])*4+1 for i in range(numcols)]
        predictions.append(predict_list)
        
    predict_sums=[]
    for i in range(len(predictions[0])):
        predict_sums.append(0)
        for j in range(len(predictions)):
            predict_sums[i] += predictions[j][i]
    
    predictions_df = pd.DataFrame({"movie_id":range(min_id,min_id+numcols),"rating":[x/len(input_list) for x in predict_sums]})
    predictions_df.query(f'movie_id not in {[x[0] for x in input_list]}', inplace=True)
    
    if title_ref is not None:
        predictions_df["title"]=[get_title(title_ref, i) for i in predictions_df["movie_id"]]
    
    return predictions_df.sort_values("rating",ascending=False)

Outputs 2D array of floats (`arraylike=False`) or lists (`arraylike=True`) as a PNG file. Used for external storage of calculated data.

In [94]:
def matrix_to_png(matrix, range_=(0,1), arraylike=True):
    num_rows = len(matrix)
    num_cols = max(len(matrix[0]), len(matrix[-1]))
    if arraylike:
        num_channels = max(len(matrix[0][0]),3)
    else:
        num_channels = 1

    img_array = np.reshape(np.array([0,0,0]*num_rows*num_cols),[num_rows, num_cols*3])
    
    def interp(x):
        return np.interp(x, range_,[0,255])
    
    for i in range(num_rows):
        num_cols_i = len(matrix[i])
        for j in range(num_cols_i):
            if not arraylike:
                for k in range(3):
                    img_array[i][3*j+k]=interp(matrix[i][j])
            else:
                for k in range(num_channels):
                    img_array[i][3*j+k]=interp(matrix[i][j][k])
                    
    img_array = np.vstack(map(np.uint8, img_array))
    
    return png.from_array(img_array,'RGB')

Reads PNG file to 2D matrix of floats (`arraylike=False`, taken from Red value of each pixel) or lists (`arraylike=True`, reads all 3 color channels). Expects 24-bit (8bpc) RGB encoding.

In [95]:
def png_to_matrix(filename, output_range=(0,1), arraylike=True):
    
    r=png.Reader(filename=filename)
    png_list = list(r.read()[2])
    
    def interp(x):
        return np.interp(x, [0,255],output_range)
    
    matrix=[]
    for i in range(len(png_list)):
        matrix.append([])
        for j in range(int(len(png_list[0])/3)):
            if arraylike:
                matrix[i].append([interp(png_list[i][3*j+k]) for k in range(3)])
            else:
                matrix[i].append(interp(png_list[i][3*j]))
    return matrix

Compute values for our test dataset

In [ ]:
means_matrix = rough_means_matrix(df, min_id, max_id, movies, low_clip=150, progress=True, prog_interval=500)

In [ ]:
matrix_to_png(means_matrix).save("means_output_3.png")

Load previously-saved data

In [96]:
means_matrix_1 = png_to_matrix("means_output_1.png")

In [97]:
user_input = [[4508,4],[4602,1],[4501,3],[4655,2]]
get_predictions(user_input, means_matrix_1, min_id, titles_df)

,movie_id,rating,title
140,4640,3.598039,Rain Man
156,4656,3.519608,Erin Brockovich
45,4545,3.329412,Dances With Wolves: Special Edition
133,4633,3.115686,Gandhi
77,4577,3.105882,Steel Magnolias
90,4590,2.764706,The Forgotten
6,4506,2.678431,Breakfast at Tiffany's
161,4661,2.515686,Deuce Bigalow: Male Gigolo
160,4660,2.470588,The Shipping News
128,4628,2.445098,Kolya


In [98]:
user_input = [[4640,4],[4633,3],[4658,5],[4670,1],[4506,3]]
get_predictions(user_input, means_matrix_1, min_id, titles_df)

,movie_id,rating,title
9,4509,3.955294,Little House on the Prairie: Season 4
163,4663,3.839216,The Wedding Banquet
45,4545,3.828235,Dances With Wolves: Special Edition
128,4628,3.774902,Kolya
22,4522,3.741961,Independent's Day
151,4651,3.713725,Goodbye
156,4656,3.691765,Erin Brockovich
77,4577,3.682353,Steel Magnolias
21,4521,3.676078,Wire in the Blood: Justice Painted Blind
153,4653,3.674510,The Burning Bed


In [99]:
means_matrix_2 = png_to_matrix("means_output_2.png")

In [100]:
user_input = [[4508,4],[4602,1],[4501,3],[4655,2]]
get_predictions(user_input, means_matrix_2, min_id, titles_df)

,movie_id,rating,title
140,4640,3.598039,Rain Man
156,4656,3.519608,Erin Brockovich
45,4545,3.329412,Dances With Wolves: Special Edition
133,4633,3.115686,Gandhi
77,4577,3.105882,Steel Magnolias
90,4590,2.764706,The Forgotten
6,4506,2.678431,Breakfast at Tiffany's
161,4661,2.515686,Deuce Bigalow: Male Gigolo
160,4660,2.470588,The Shipping News
128,4628,2.415686,Kolya


In [101]:
user_input = [[4640,4],[4633,3],[4658,5],[4670,1],[4506,3]]
get_predictions(user_input, means_matrix_2, min_id, titles_df)

,movie_id,rating,title
9,4509,3.955294,Little House on the Prairie: Season 4
163,4663,3.839216,The Wedding Banquet
45,4545,3.828235,Dances With Wolves: Special Edition
128,4628,3.774902,Kolya
22,4522,3.741961,Independent's Day
151,4651,3.713725,Goodbye
156,4656,3.691765,Erin Brockovich
77,4577,3.682353,Steel Magnolias
21,4521,3.676078,Wire in the Blood: Justice Painted Blind
153,4653,3.674510,The Burning Bed


In [102]:
views_matrix_ = views_matrix(df, min_id, max_id, movies, True, 500)

0 records complete... (0.0 s elapsed)
500 records complete... (0.91 s elapsed)
1000 records complete... (1.71 s elapsed)
1500 records complete... (2.57 s elapsed)
2000 records complete... (3.35 s elapsed)
2500 records complete... (4.31 s elapsed)
3000 records complete... (5.33 s elapsed)
3500 records complete... (6.21 s elapsed)
4000 records complete... (7.08 s elapsed)
4500 records complete... (8.13 s elapsed)
5000 records complete... (9.15 s elapsed)
5500 records complete... (9.96 s elapsed)
6000 records complete... (10.79 s elapsed)
6500 records complete... (11.7 s elapsed)
7000 records complete... (12.51 s elapsed)
7500 records complete... (13.29 s elapsed)
8000 records complete... (14.1 s elapsed)
8500 records complete... (15.09 s elapsed)
9000 records complete... (16.08 s elapsed)
9500 records complete... (16.94 s elapsed)
10000 records complete... (18.28 s elapsed)
10500 records complete... (19.15 s elapsed)
11000 records complete... (20.11 s elapsed)
11500 records complete... (

In [103]:
user_input = [[4640,4],[4633,3],[4658,5],[4670,1],[4506,3]]
get_predictions_clip(user_input, means_matrix_1, min_id, views_matrix_, 200, titles_df).head(10)

,movie_id,rating,title
9,4509,3.955294,Little House on the Prairie: Season 4
163,4663,3.839216,The Wedding Banquet
45,4545,3.828235,Dances With Wolves: Special Edition
128,4628,3.774902,Kolya
22,4522,3.741961,Independent's Day
151,4651,3.713725,Goodbye
156,4656,3.691765,Erin Brockovich
77,4577,3.682353,Steel Magnolias
153,4653,3.674510,The Burning Bed
65,4565,3.607059,The Paper Chase


In [104]:
user_input = [[4508,4],[4602,1],[4501,3],[4655,2],[4640,5]]
get_predictions_clip(user_input, means_matrix_1, min_id, views_matrix_, 200, titles_df).head(10)

,movie_id,rating,title
156,4656,3.602353,Erin Brockovich
77,4577,2.992157,Steel Magnolias
90,4590,2.636078,The Forgotten
45,4545,2.615686,Dances With Wolves: Special Edition
133,4633,2.240784,Gandhi
158,4658,2.206275,Seabiscuit: American Experience
163,4663,2.198431,The Wedding Banquet
9,4509,2.193725,Little House on the Prairie: Season 4
75,4575,2.168627,Scooby-Doo Where Are You?: Seasons 1 & 2
128,4628,2.160784,Kolya
